In [6]:
from transformers import BertTokenizer, BertModel

# 加载分词器
tokenizer = BertTokenizer.from_pretrained('bert')

# 加载模型
model = BertModel.from_pretrained('bert')


In [50]:
import pandas as pd

# 读取CSV文件到DataFrame
df = pd.read_csv("paragraph.csv")


In [46]:
import nltk
from nltk.tokenize import sent_tokenize

nltk.data.path.append("nltk_data")

# 使用nltk将段落拆分为句子
sentences = sent_tokenize(paragraph)
# 输出分割后的句子
for sentence in sentences:
    print(sentence)


,Sentence Number,Page,Sentence
0,1,1,Editor: Ivan Titov\n\nAbstract\n\nTransfer lea...
1,2,1,The effectiveness of transfer learning has giv...
2,3,1,"In this paper, we explore the landscape of tra..."
3,4,1,Our systematic study compares pre-training obj...
4,5,1,By combining the insights from our exploration...
...,...,...,...
108,23,5,2.2.
109,24,5,The Colossal Clean Crawled Corpus\n\nMuch of t...
110,25,5,"In this paper, we are interested in measuring ..."
111,26,5,"To generate data sets that satisfy our needs, ..."


In [63]:
# 初始化全局句子计数器
global_sentence_number = 1

# 初始化全局的句子列表
sentences = []
sentence_data = []

# 遍历每一行，获取页码和内容
for index, row in df.iterrows():
    page = row['page']  # 获取所在的页码
    paragraph = row['content']  # 获取段落内容
    sentence_list = sent_tokenize(paragraph)  # 将段落分割为句子
    
    # 保存每个句子的编号和所在页码，不重置sentence_number
    for sentence in sentence_list:
        sentence_data.append({'Sentence Number': global_sentence_number, 'Page': page, 'Sentence': sentence})
        sentences.append (sentence)
        global_sentence_number += 1  # 增加全局句子编号

sentence_df = pd.DataFrame(sentence_data)
# 输出句子变量
sentence_df
print (sentences)

['Editor: Ivan Titov\n\nAbstract\n\nTransfer learning, where a model is first pre-trained on a data-rich task before being fine- tuned on a downstream task, has emerged as a powerful technique in natural language processing (NLP).', 'The effectiveness of transfer learning has given rise to a diversity of approaches, methodology, and practice.', 'In this paper, we explore the landscape of transfer learning techniques for NLP by introducing a unified framework that converts all text-based language problems into a text-to-text format.', 'Our systematic study compares pre-training objectives, architectures, unlabeled data sets, transfer approaches, and other factors on dozens of language understanding tasks.', 'By combining the insights from our exploration with scale and our new “Colossal Clean Crawled Corpus”, we achieve state-of-the-art results on many benchmarks covering summarization, question answering, text classification, and more.', 'To facilitate future work on transfer learning 

In [56]:
import nltk
from nltk.tokenize import sent_tokenize

# 确保nltk数据路径正确
nltk.data.path.append("nltk_data")

# 创建用于存储每个句子的编号和所在的页码的数据
sentence_data = []

# 初始化全局句子计数器
global_sentence_number = 1

# 遍历每一行，获取页码和内容
for index, row in df.iterrows():
    page = row['page']  # 获取所在的页码
    paragraph = row['content']  # 获取段落内容
    sentences = sent_tokenize(paragraph)  # 将段落分割为句子
    
    # 保存每个句子的编号和所在页码，不重置sentence_number
    for sentence in sentences:
        sentence_data.append({'Sentence Number': global_sentence_number, 'Page': page, 'Sentence': sentence})
        global_sentence_number += 1  # 增加全局句子编号

# 创建新的DataFrame
sentence_df = pd.DataFrame(sentence_data)

# 输出结果
sentence_df


,Sentence Number,Page,Sentence
0,1,1,Editor: Ivan Titov\n\nAbstract\n\nTransfer lea...
1,2,1,The effectiveness of transfer learning has giv...
2,3,1,"In this paper, we explore the landscape of tra..."
3,4,1,Our systematic study compares pre-training obj...
4,5,1,By combining the insights from our exploration...
...,...,...,...
108,109,5,2.2.
109,110,5,The Colossal Clean Crawled Corpus\n\nMuch of t...
110,111,5,"In this paper, we are interested in measuring ..."
111,112,5,"To generate data sets that satisfy our needs, ..."


In [58]:
len(sentences)

113

In [64]:

# 动态摘要的长度阈值
max_char_length = 3000

# 初始化变量
current_chunk = ""
summaries = []
flag = 1
# 动态汇总句子，按需生成摘要
for sentence in sentences:
    if len(current_chunk) + len(sentence) <= max_char_length:
        # 如果当前汇总长度小于阈值，继续累加句子
        current_chunk += " " + sentence
    else:
        # 如果超过阈值，对当前累加的文本进行摘要
        print (current_chunk.strip())
        print (flag)
        flag += 1
        summary = summarizer(current_chunk.strip(), min_length=25, do_sample=False)
        summaries.append(summary[0]['summary_text'])
        
        # 重置current_chunk并添加当前句子
        current_chunk = sentence

# 处理最后剩下的文本
if current_chunk:
    summary = summarizer(current_chunk.strip(), min_length=25, do_sample=False)
    summaries.append(summary[0]['summary_text'])

# 合并所有摘要
final_summary = " ".join(summaries)
print("Final Summary:", final_summary)

Editor: Ivan Titov

Abstract

Transfer learning, where a model is first pre-trained on a data-rich task before being fine- tuned on a downstream task, has emerged as a powerful technique in natural language processing (NLP). The effectiveness of transfer learning has given rise to a diversity of approaches, methodology, and practice. In this paper, we explore the landscape of transfer learning techniques for NLP by introducing a unified framework that converts all text-based language problems into a text-to-text format. Our systematic study compares pre-training objectives, architectures, unlabeled data sets, transfer approaches, and other factors on dozens of language understanding tasks. By combining the insights from our exploration with scale and our new “Colossal Clean Crawled Corpus”, we achieve state-of-the-art results on many benchmarks covering summarization, question answering, text classification, and more. To facilitate future work on transfer learning for NLP, we release o

In [65]:
summary_text = summarizer(final_summary, min_length=25, do_sample=False)
summary_text = summary_text[0]['summary_text']
print (summary_text)

Transfer learning is where a model is first pre-trained on a data-rich task before being fine- tuned on a downstream task. Modern techniques for transfer learning in NLP often pre-train using unsupervised learning on unlabeled data. The basic idea underlying our work is to treat every text processing problem as a “text-to-text” problem.


In [66]:
from transformers import pipeline
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 使用本地预训练的BERT模型进行句子嵌入
nlp = pipeline("feature-extraction", model="bert", device=-1)

# 获取每个句子的嵌入向量
embeddings = [nlp(sentence)[0] for sentence in sentences]

# 使用第一个token [CLS] 的嵌入作为句子的表示
embeddings = [np.mean(np.array(embedding), axis=0) for embedding in embeddings]

# 获取摘要文本的嵌入向量，并将其作为参考
summary_embedding = np.mean(np.array(nlp(summary_text)[0]), axis=0)

# 计算摘要嵌入与其他句子嵌入的相似度
similarities = cosine_similarity([summary_embedding], embeddings)[0]

# 输出相似度
print("Similarities:", similarities)

# 排序句子
ranked_sentences = [x for _, x in sorted(zip(similarities, sentences), reverse=True)]

print("Ranked Sentences by Importance:")
for sentence in ranked_sentences:
    print(sentence)


Similarities: [0.93650322 0.78242584 0.85912806 0.88297509 0.87566812 0.85428819
 0.8818037  0.89396976 0.88218859 0.75271964 0.48903396 0.55948072
 0.72512338 0.6374271  0.82419598 0.67783637 0.720643   0.81283304
 0.7351051  0.90876961 0.88695824 0.79361174 0.83870122 0.84022429
 0.85525485 0.94326148 0.81206557 0.90192953 0.85364455 0.81657433
 0.83003319 0.84945429 0.80336911 0.88318582 0.83216558 0.88572691
 0.69976192 0.88677169 0.83823022 0.78934714 0.68123013 0.75342037
 0.75341415 0.63034269 0.6445636  0.81377113 0.91047072 0.83829957
 0.72409427 0.78983854 0.73956224 0.79090853 0.90623361 0.77370946
 0.82197614 0.86380797 0.86508522 0.87614359 0.85876717 0.85446569
 0.80116385 0.73638973 0.46282405 0.62521984 0.86166114 0.91295058
 0.78926299 0.61278992 0.86801619 0.81687977 0.72335021 0.77853005
 0.81448764 0.78834593 0.8347363  0.82659516 0.87673581 0.79641385
 0.8668228  0.71777374 0.82573791 0.77770906 0.83446182 0.83221291
 0.86032051 0.81544302 0.8089351  0.8169563  0.7

In [67]:
sentence_df['Similarity'] = similarities
sentence_df

,Sentence Number,Page,Sentence,Similarity
0,1,1,Editor: Ivan Titov\n\nAbstract\n\nTransfer lea...,0.936503
1,2,1,The effectiveness of transfer learning has giv...,0.782426
2,3,1,"In this paper, we explore the landscape of tra...",0.859128
3,4,1,Our systematic study compares pre-training obj...,0.882975
4,5,1,By combining the insights from our exploration...,0.875668
...,...,...,...,...
108,109,5,2.2.,0.545220
109,110,5,The Colossal Clean Crawled Corpus\n\nMuch of t...,0.899259
110,111,5,"In this paper, we are interested in measuring ...",0.821298
111,112,5,"To generate data sets that satisfy our needs, ...",0.846122


In [68]:
from sklearn.cluster import MiniBatchKMeans
import numpy as np

# 使用 MiniBatchKMeans 进行聚类
n_clusters = 5  # 你想要的聚类数量
mini_kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=0, batch_size=10)
cluster_labels = mini_kmeans.fit_predict(similarities.reshape(-1, 1))

# 计算每个聚类的相似度平均值
cluster_similarities = {i: [] for i in range(n_clusters)}
clustered_sentences = {i: [] for i in range(n_clusters)}  # 存储每个簇的句子

# 将句子和相似度根据聚类标签分组
for label, similarity, sentence in zip(cluster_labels, similarities, sentences):
    cluster_similarities[label].append(similarity)
    clustered_sentences[label].append(sentence)

# 输出每个聚类的平均相似度
cluster_avg_similarities = {cluster: np.mean(sim_values) for cluster, sim_values in cluster_similarities.items()}

# 对聚类按平均相似度排序
sorted_clusters = sorted(cluster_avg_similarities.items(), key=lambda x: x[1], reverse=True)
sorted_cluster_mapping = {old_label: new_label for new_label, (old_label, _) in enumerate(sorted_clusters)}

# 更新 cluster_labels 为新的顺序编码
new_cluster_labels = [sorted_cluster_mapping[label] for label in cluster_labels]

# 将句子按新的簇标签分组
new_clustered_sentences = {i: [] for i in range(n_clusters)}
for new_label, sentence in zip(new_cluster_labels, sentences):
    new_clustered_sentences[new_label].append(sentence)

# 按新的簇标签输出排序后的句子
# 输出排序后每个聚类的平均相似度和句子
print("\nClustered Sentences by Sorted Average Similarity:")
for new_label, (old_label, avg_similarity) in enumerate(sorted_clusters):
    print(f"\nCluster {new_label} (Old Cluster {old_label}) - Average Similarity: {avg_similarity:.4f}")
    clustered_group = new_clustered_sentences[new_label]
    print(f"Size: {len(clustered_group)}")
    for sentence in clustered_group:
        print(sentence)





Clustered Sentences by Sorted Average Similarity:

Cluster 0 (Old Cluster 4) - Average Similarity: 0.8862
Size: 29
Editor: Ivan Titov

Abstract

Transfer learning, where a model is first pre-trained on a data-rich task before being fine- tuned on a downstream task, has emerged as a powerful technique in natural language processing (NLP).
In this paper, we explore the landscape of transfer learning techniques for NLP by introducing a unified framework that converts all text-based language problems into a text-to-text format.
Our systematic study compares pre-training objectives, architectures, unlabeled data sets, transfer approaches, and other factors on dozens of language understanding tasks.
By combining the insights from our exploration with scale and our new “Colossal Clean Crawled Corpus”, we achieve state-of-the-art results on many benchmarks covering summarization, question answering, text classification, and more.
1

Keywords: transfer learning, natural language processing, mu

In [69]:
sentence_df['Cluster Number'] = new_cluster_labels
sentence_df

,Sentence Number,Page,Sentence,Similarity,Cluster Number
0,1,1,Editor: Ivan Titov\n\nAbstract\n\nTransfer lea...,0.936503,0
1,2,1,The effectiveness of transfer learning has giv...,0.782426,2
2,3,1,"In this paper, we explore the landscape of tra...",0.859128,0
3,4,1,Our systematic study compares pre-training obj...,0.882975,0
4,5,1,By combining the insights from our exploration...,0.875668,0
...,...,...,...,...,...
108,109,5,2.2.,0.545220,4
109,110,5,The Colossal Clean Crawled Corpus\n\nMuch of t...,0.899259,0
110,111,5,"In this paper, we are interested in measuring ...",0.821298,1
111,112,5,"To generate data sets that satisfy our needs, ...",0.846122,1


In [70]:
sentence_df.to_csv('clustered_sentences.csv', index=False, encoding='utf-8')

In [72]:
# Convert the dataframe to JSON format for use in the HTML page
json_data = sentence_df.to_json(orient='records')

# Save the JSON data to a file so it can be used later
json_file_path = 'clustered_sentences.json'
with open(json_file_path, 'w') as f:
    f.write(json_data)

json_file_path  # Return the path of the saved JSON file


'clustered_sentences.json'